In [1]:
import math as m
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
def W(n,N):
    return [m.cos(2*m.pi*n/N),m.sin(2*m.pi*n/N)]

In [3]:
def add(a,b):
    x = a[0]+b[0]
    y = a[1]+b[1]
    return [x,y]

def sub(a,b):
    x = a[0]-b[0]
    y = a[1]-b[1]
    return [x,y]

def mult(a,b):
    x = a[0]*b[0]-a[1]*b[1]
    y = a[0]*b[1]+a[1]*b[0]
    return [x,y]

In [4]:
def Signal(inp1,inp2):
    a = []
    b = []
    a = list(str(inp1))
    b = list(str(inp2))
    for i in range(len(a)):
        a[i] = [int(a[i]),0]
    for i in range(len(b)):
        b[i] = [int(b[i]),0]
    for _ in range(len(b)-1):
        a.append([0,0])
    for _ in range(len(a)-len(b)):
        b.append([0,0])
    #padding
    padding = int(1<<int(m.log2(len(a))+1))-len(a)
    for i in range(padding):
        a.append([0,0])
        b.append([0,0])
    return a,b,padding

In [5]:
def FFT(signal):  # the passed signal should be complex [x,y]
    if len(signal)==2:
        temp = signal[1] #mult(W(0,len(signal)),signal[1])
        return [add(signal[0],temp),sub(signal[0],temp)]
    else:
        even = [signal[i] for i in range(0,len(signal),2)]
        odd = [signal[i] for i in range(1,len(signal),2)]
        e,o = FFT(even),FFT(odd)
        transform = []
        o = [mult(W(-i,len(signal)),o[i]) for i in [*range(len(o))]]
        for i in range(len(signal)//2):
            transform.append(add(e[i],o[i]))
        for i in range(len(signal)//2):
            transform.append(sub(e[i],o[i]))
        return transform

In [6]:
def karatsuba(x,y):
    if x<10 or y<10:
        return x*y
    else:
        n = max(len(str(x)),len(str(y)))
        half = n//2
        a = x//(10**half)
        b = x%(10**half)
        c = y//(10**half)
        d = y%(10**half)
        ac = karatsuba(a,c)
        bd = karatsuba(b,d)
        ac_plus_bd = karatsuba(a+b,c+d)-ac-bd
        return ac*(10**(2*half))+ac_plus_bd*(10**half)+bd

In [7]:
def FFTmultiply(f,g):
    f,g,padding = Signal(f,g)
    F = FFT(f)
    G = FFT(g)
    
    FxG = []
    for i in range(len(F)):
        FxG.append(mult(F[i],G[i]))
    
    f_g = FFT(FxG)
    f_g.append(f_g.pop(0))
    
    sum = 0
    for i in range(len(f_g)):
        sum += int(f_g[i][0]*10**i)/10**padding
    
    return int(sum)//len(f_g)

In [8]:
def multiply(a,b):
    a = str(a)[::-1]
    b = str(b)[::-1]
    sum = 0
    for i,x in enumerate(a):
        for j,y in enumerate(b):
            sum += int(x)*int(y)*10**i*10**j
    return sum

In [9]:
%%time

karatsuba(199384721198273849919938472119827384993841993847213847211982738438472119827387211982738438472119827384,241123119938472119819938411982738438472119827382738499384721198273842192893842411231219289384)


CPU times: user 6.55 ms, sys: 0 ns, total: 6.55 ms
Wall time: 6.46 ms


48076266043390210071484935435213665103043245028169028724497894758308098793888271555800022328205895830413643880223549217978518547577856135390094915066801102991694571848679935683736007825223691456

In [10]:
%%time

FFTmultiply(199384721198273849919938472119827384993841993847213847211982738438472119827387211982738438472119827384,241123119938472119819938411982738438472119827382738499384721198273842192893842411231219289384)


CPU times: user 15.2 ms, sys: 0 ns, total: 15.2 ms
Wall time: 14.5 ms


48076266043388573486387020254084489829912201931951939606321105549472905427581425320729679119333190667070055666628234717107566464981964163548282857328268896420642568182067513733314763268999348224

In [11]:
%%time

multiply(199384721198273849919938472119827384993841993847213847211982738438472119827387211982738438472119827384,241123119938472119819938411982738438472119827382738499384721198273842192893842411231219289384)


CPU times: user 30 ms, sys: 557 µs, total: 30.6 ms
Wall time: 29.2 ms


48076266043390210071484935435213665103043245028169028724497894758308098793888271555800022328205895830413643880223549217978518547577856135390094915066801102991694571848679935683736007825223691456

In [12]:
1231198279124*121987987934913

150191400819264160118656212

In [13]:
FFTmultiply(1231198279124,121987987934913)

150191400819257542837272576

In [14]:
def NPmultiply(f,g):

    f,g,padding = Signal(f,g)
    f = [x[0] for x in f[:-padding-1]]
    g = [x[0] for x in g[:-padding-1]]
    F = np.fft.fft(f)
    G = np.fft.fft(f)
    
    FxG = F*G
    
    f_g = np.flip(np.fft.ifft(FxG).real)
    sum = 0
    for i in range(len(f_g)):
        sum += f_g[i]*10**i
    
    return sum

In [15]:
NPmultiply(11231231,1232)

18797505496.0